# Keras Train BNN Classifier for FANET Reliability
Date: 18/04/2023
Desc: To train a BNN classifier to predict FANET reliability and failure modes

## Imports

In [1]:
import pandas as pd
import numpy as np 
import sklearn
import os

# Import necessary modules
from sklearn.model_selection import train_test_split

# Keras specific
import tensorflow as tf
import tensorflow_probability as tfp
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense, Input
from keras.utils import to_categorical 
import pickle

2023-04-19 10:36:52.415587: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-19 10:36:52.527683: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-19 10:36:52.532306: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-19 10:36:52.532320: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

## Set Params and Load Dataset

In [7]:
# Training params
EPOCHS = 30
TEST_SPLIT = 0.10 # Test split percentage
checkpoint_filepath = '/home/research-student/omnet-fanet/nn_checkpoints/bnn_v2_hovering_novideo_sinr'
delay_threshold = 1 # NOTE: REMEMBER TO SET DELAY THRESHOLD
# checkpoint_filepath = "/home/research-student/omnet-fanet/nn_checkpoints"

# Compile all data
# dl_df_8uav = pd.read_hdf("/home/rlim0005/FANET_Dataset/Dataset_NP10000_BPSK_6-5Mbps/Dataset_NP10000_BPSK_6-5Mbps_8UAVs_processed_downlink.h5", '8_UAVs')
# dl_df_16uav = pd.read_hdf("/home/rlim0005/FANET_Dataset/Dataset_NP10000_BPSK_6-5Mbps/Dataset_NP10000_BPSK_6-5Mbps_16UAVs_processed_downlink.h5", '16_UAVs')
# dl_df_24uav = pd.read_hdf("/home/rlim0005/FANET_Dataset/Dataset_NP10000_BPSK_6-5Mbps/Dataset_NP10000_BPSK_6-5Mbps_24UAVs_processed_downlink.h5", '24_UAVs')
# dl_df_32uav = pd.read_hdf("/home/rlim0005/FANET_Dataset/Dataset_NP10000_BPSK_6-5Mbps/Dataset_NP10000_BPSK_6-5Mbps_32UAVs_processed_downlink.h5", '32_UAVs')
# dl_df_40uav = pd.read_hdf("/home/rlim0005/FANET_Dataset/Dataset_NP10000_BPSK_6-5Mbps/Dataset_NP10000_BPSK_6-5Mbps_40UAVs_processed_downlink.h5", '40_UAVs')
# dl_df = pd.concat([dl_df_8uav, dl_df_16uav, dl_df_24uav, dl_df_32uav, dl_df_40uav], ignore_index=True)

dl_df = pd.read_csv("/media/research-student/One Touch/FANET Datasets/Dataset_NP10000_64QAM_65Mbps_Hovering_NoVideo/8UAVs_Exp1_processed_downlink.csv")

dl_df.sort_values(by = "U2G_H_Dist")

data_df = dl_df[["Mean_SINR", "Std_Dev_SINR", "Num_Members", "Bytes", "UAV_Sending_Interval", "Incorrectly_Received", "Queue_Overflow"]].copy()
data_df["Reliable"] = np.where(dl_df['Packet_State'] == "Reliable" , 1, 0)
data_df["Delay_Exceeded"] = np.where(dl_df['Delay'] > delay_threshold, 1, 0)

## Preprocess Data

In [8]:
# Normalize data
max_mean_sinr = 521 # The max mean SINR calculated at (50,60) is 520.2907250903191
max_std_dev_sinr = 252 # The max std dev SINR calculated at (50,60) is 251.44889082897834
max_num_members = 39
max_bytes = 1500 # Max Ethernet MTU
max_uav_send_int = 1000
data_df["Mean_SINR"] = data_df["Mean_SINR"].div(max_mean_sinr)
data_df["Std_Dev_SINR"] = data_df["Std_Dev_SINR"].div(max_std_dev_sinr)
data_df["Num_Members"] = data_df["Num_Members"].div(max_num_members)
data_df["Bytes"] = data_df["Bytes"].div(max_bytes)
data_df["UAV_Sending_Interval"] = data_df["UAV_Sending_Interval"].div(max_uav_send_int)

# Split to train and test
data_df_train, data_df_test = train_test_split(data_df, test_size=TEST_SPLIT, random_state=40, shuffle=False)
X_train = data_df_train[["Mean_SINR", "Std_Dev_SINR", "Num_Members", "Bytes", "UAV_Sending_Interval"]].values
X_test = data_df_test[["Mean_SINR", "Std_Dev_SINR", "Num_Members", "Bytes", "UAV_Sending_Interval"]].values
X_train_all = data_df[["Mean_SINR", "Std_Dev_SINR", "Num_Members", "Bytes", "UAV_Sending_Interval"]].values
reliability_train = data_df_train["Reliable"].values
reliability_test = data_df_test["Reliable"].values
reliability_train_all = data_df["Reliable"].values
incr_rcvd_train = data_df_train["Incorrectly_Received"].values
incr_rcvd_test = data_df_test["Incorrectly_Received"].values
incr_rcvd_train_all = data_df["Incorrectly_Received"].values
delay_excd_train = data_df_train["Delay_Exceeded"].values
delay_excd_test = data_df_test["Delay_Exceeded"].values
delay_excd_train_all = data_df["Delay_Exceeded"].values
queue_overflow_train = data_df_train["Queue_Overflow"].values
queue_overflow_test = data_df_test["Queue_Overflow"].values
queue_overflow_train_all = data_df["Queue_Overflow"].values

reliability_train = to_categorical(reliability_train) 
reliability_test = to_categorical(reliability_test)
reliability_train_all = to_categorical(reliability_train_all) 
incr_rcvd_train = to_categorical(incr_rcvd_train) 
incr_rcvd_test = to_categorical(incr_rcvd_test)
incr_rcvd_train_all = to_categorical(incr_rcvd_train_all) 
delay_excd_train = to_categorical(delay_excd_train) 
delay_excd_test = to_categorical(delay_excd_test)
delay_excd_train_all = to_categorical(delay_excd_train_all)
queue_overflow_train = to_categorical(queue_overflow_train) 
queue_overflow_test = to_categorical(queue_overflow_test)
queue_overflow_train_all = to_categorical(queue_overflow_train_all)

In [9]:
len(X_train_all)

216456

## Train BNN Model

In [10]:
# KL Divergence loss function
# Ref: https://colab.research.google.com/github/tensorchiefs/dl_book/blob/master/chapter_08/nb_ch08_03.ipynb#scrollTo=if4YhGgNzqnv
# Ref: https://github.com/tensorflow/probability/blob/main/tensorflow_probability/examples/bayesian_neural_network.py
kernel_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (len(X_train_all))
bias_divergence_fn=lambda q, p, _: tfp.distributions.kl_divergence(q, p) / (len(X_train_all))

inputs = Input(shape=(5,))
base = tfp.layers.DenseFlipout(50, kernel_divergence_fn=kernel_divergence_fn, bias_divergence_fn=bias_divergence_fn, activation='relu')(inputs)
base = tfp.layers.DenseFlipout(25, kernel_divergence_fn=kernel_divergence_fn, bias_divergence_fn=bias_divergence_fn, activation='relu')(base)
base = tfp.layers.DenseFlipout(10, kernel_divergence_fn=kernel_divergence_fn, bias_divergence_fn=bias_divergence_fn, activation='relu')(base)
reliability_hl = tfp.layers.DenseFlipout(10, kernel_divergence_fn=kernel_divergence_fn, bias_divergence_fn=bias_divergence_fn, activation='relu')(base)
incr_rcvd_out_hl = tfp.layers.DenseFlipout(10, kernel_divergence_fn=kernel_divergence_fn, bias_divergence_fn=bias_divergence_fn, activation='relu')(base)
delay_excd_hl = tfp.layers.DenseFlipout(10, kernel_divergence_fn=kernel_divergence_fn, bias_divergence_fn=bias_divergence_fn, activation='relu')(base)
queue_overflow_hl = tfp.layers.DenseFlipout(10, kernel_divergence_fn=kernel_divergence_fn, bias_divergence_fn=bias_divergence_fn, activation='relu')(base)
reliability_out = tfp.layers.DenseFlipout(2, kernel_divergence_fn=kernel_divergence_fn, bias_divergence_fn=bias_divergence_fn, activation='softmax', name='reliability')(reliability_hl)
incr_rcvd_out = tfp.layers.DenseFlipout(8, kernel_divergence_fn=kernel_divergence_fn, bias_divergence_fn=bias_divergence_fn, activation='softmax', name='incorrectly_received')(incr_rcvd_out_hl)
delay_excd_out = tfp.layers.DenseFlipout(2, kernel_divergence_fn=kernel_divergence_fn, bias_divergence_fn=bias_divergence_fn, activation='softmax', name='delay_exceeded')(delay_excd_hl)
queue_overflow_out = tfp.layers.DenseFlipout(2, kernel_divergence_fn=kernel_divergence_fn, bias_divergence_fn=bias_divergence_fn, activation='softmax', name='queue_overflow')(queue_overflow_hl)
model = Model(inputs=inputs, outputs = [reliability_out, incr_rcvd_out, delay_excd_out, queue_overflow_out])

# Compile the model
'''
The loss functions are crossentropy, for classification. 
The Keras API will then automatically add the Kullback-Leibler divergence (contained on the individual layers of
the model), to the cross entropy loss, effectively calcuating the (negated) Evidence Lower Bound Loss (ELBO).
Ref: https://github.com/tensorflow/probability/blob/main/tensorflow_probability/examples/bayesian_neural_network.py
'''
model.compile(optimizer='adam', 
              loss={'reliability': 'binary_crossentropy',
                    'incorrectly_received': 'categorical_crossentropy',
                    'delay_exceeded': 'binary_crossentropy',
                    'queue_overflow': 'binary_crossentropy'},
              metrics={'reliability': 'accuracy',
                    'incorrectly_received': 'accuracy',
                    'delay_exceeded': 'accuracy',
                    'queue_overflow': 'accuracy'},)

model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpoint_filepath,"model.{epoch:03d}-{val_loss:.4f}.h5"),
    save_weights_only=True,
    monitor='val_loss',
    mode='auto',
    save_freq='epoch')

# Y_train = [reliability_train, incr_rcvd_train, delay_excd_train, queue_overflow_train]
Y_train_all = [reliability_train_all, incr_rcvd_train_all, delay_excd_train_all, queue_overflow_train_all]
Y_test = [reliability_test, incr_rcvd_test, delay_excd_test, queue_overflow_test]
history = model.fit(X_train_all, Y_train_all, epochs=EPOCHS, callbacks=[model_checkpoint_callback], validation_data=(X_test, Y_test))
with open(os.path.join(checkpoint_filepath, 'trainHistoryDict_19042023'), 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

# Save final model
model.save(os.path.join(checkpoint_filepath,"final_model.h5"))

/home/research-student/.local/lib/python3.8/site-packages/tensorflow_probability/python/layers/util.py:95: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  loc = add_variable_fn(
/home/research-student/.local/lib/python3.8/site-packages/tensorflow_probability/python/layers/util.py:105: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  untransformed_scale = add_variable_fn(


Epoch 1/30
6765/6765 [==============================] - 51s 7ms/step - loss: 0.9198 - reliability_loss: 0.0725 - incorrectly_received_loss: 0.4144 - delay_exceeded_loss: 0.1824 - queue_overflow_loss: 0.2265 - reliability_accuracy: 0.9703 - incorrectly_received_accuracy: 0.9065 - delay_exceeded_accuracy: 0.9545 - queue_overflow_accuracy: 0.9245 - val_loss: 0.6039 - val_reliability_loss: 0.0017 - val_incorrectly_received_loss: 0.2184 - val_delay_exceeded_loss: 0.1797 - val_queue_overflow_loss: 0.1808 - val_reliability_accuracy: 0.9999 - val_incorrectly_received_accuracy: 0.9563 - val_delay_exceeded_accuracy: 0.9568 - val_queue_overflow_accuracy: 0.9563
Epoch 2/30
6765/6765 [==============================] - 46s 7ms/step - loss: 0.8356 - reliability_loss: 0.0472 - incorrectly_received_loss: 0.3853 - delay_exceeded_loss: 0.1746 - queue_overflow_loss: 0.2058 - reliability_accuracy: 0.9828 - incorrectly_received_accuracy: 0.9068 - delay_exceeded_accuracy: 0.9545 - queue_overflow_accuracy: 0.